In [11]:
import torch
import pandas  as pd

from lime.lime_text import LimeTextExplainer

from transformers import RobertaTokenizer
import os
import pickle
import eli5

from transformers import T5ForConditionalGeneration, RobertaTokenizer
import torch
import pandas as pd
import torch.nn.functional as F
import numpy as np

In [12]:
def predict_example(test_comment_list):

    USE_CUDA = torch.cuda.is_available()

    net = CodeBert()
    net.load_state_dict(torch.load(model_path))
    if(USE_CUDA):
        net.cuda()
    # tokenize
    tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
    result_comments_id = tokenizer(test_comment_list,
                                    padding=True,
                                    truncation=True,
                                    max_length=120,
                                    return_tensors='pt')
    tokenizer_id = result_comments_id['input_ids']
    inputs = tokenizer_id
    batch_size = inputs.size(0)
    # initialize hidden state

    if(USE_CUDA):
        inputs = inputs.cuda()

    net.eval()
    with torch.no_grad():
        # get the output from the model
        output= net(inputs)
        output=torch.nn.Softmax(dim=1)(output)
        return output.cpu().detach().numpy()
        

In [13]:
batch_size = 32
model_path = "../models/bert_model"
bert_model_class = "microsoft/codebert-base"
test_dataset_path = '../data/clean_data/test.csv'

label_names = ['negative', 'positive']

In [32]:
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler, MaskingTextSamplers

explainer = TextExplainer(n_samples = 50, random_state=2022, sampler=MaskingTextSampler())

In [16]:
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")
model_path = "../models/pytorch_model.bin"
model.load_state_dict(torch.load(model_path))

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")

test = pd.read_csv("../data/clean_data/test.csv")
k=11
example0 = test.iloc[k*2]['new_code']
example1 = test.iloc[k*2+1]['new_code']


In [17]:
def prediction(example_list):
    logits=[]
    for example in example_list:
        input_ids = tokenizer(example, truncation=True, return_tensors="pt").input_ids
        generated_ids = model.generate(input_ids, max_length=512, output_scores=True, return_dict_in_generate=True)
        logit = generated_ids.scores[1][0][20:22]
        logit = F.softmax(logit, -1)
        p = logit.tolist()[0]
        logit = [p,1-p]
        logits.append(logit)
    return np.array(logits)

In [33]:
explainer.fit(example0, prediction)
explainer.show_prediction(target_names=['negative', 'positive'])

D:\Users\Cryst\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [34]:
explainer.show_weights(top=200)

D:\Users\Cryst\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+0.673,from
+0.500,from sqlalchemy
+0.368,fetchall
+0.274,str
+0.267,pymysql
+0.222,on ua
+0.222,ua on
+0.222,on
+0.217,ua
+0.211,text from


In [35]:
explainer.fit(example1, prediction)
explainer.show_prediction(target_names=['negative', 'positive'])

D:\Users\Cryst\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [36]:
explainer.show_weights(top=200)

D:\Users\Cryst\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+0.840,bind engine
+0.794,id
+0.727,def
+0.713,create_engine
+0.598,0
+0.464,sessionmaker bind
+0.455,join
+0.423,bind
+0.407,mysql
+0.374,session
